##CV features

In [3]:
import re
import logging 
import gensim
import pandas as pd
import numpy as np
%matplotlib inline
import seaborn as sns
import xgboost as xgb 
from time import time 
from tqdm import tqdm 
from functools import reduce
from sklearn.manifold import TSNE
from collections import defaultdict
from gensim.models import FastText
from sklearn.metrics import r2_score
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

C:\Users\jyoti\anaconda3\envs\nlp\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [4]:
path='C:\\Users\\jyoti\\Desktop\\NLP\\NLP-project-\\'

In [5]:
import pandas as pd
df = pd.read_csv(path+ "Dataset\\df_final_cv.csv")

C:\Users\jyoti\anaconda3\envs\nlp\lib\site-packages\IPython\core\interactiveshell.py:3229: DtypeWarning: Columns (49) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
df

,Unnamed: 0,status,description,username,fbid,connected_fb_page,profile_pic_url,full_name,is_private,category_name,...,input_stop_id,warning,error,tags,confidence_score,accent_color,is_bw,dominant_colors,bg_color,fore_color
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,"['text', 'font', 'graphics', 'screenshot', 'gr...","[0.9980798959732056, 0.9481294751167297, 0.881...","(0.788235294117647, 0.00784313725490196, 0.007...",0,"[(1.0, 0.7529411764705882, 0.796078431372549)]","(1.0, 0.7529411764705882, 0.796078431372549)","(1.0, 0.7529411764705882, 0.796078431372549)"
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,"['dessert', 'baked goods', 'baking', 'snack', ...","[0.9897554516792297, 0.987897515296936, 0.9828...","(0.6901960784313725, 0.14901960784313725, 0.10...",0,"[(0.6470588235294118, 0.16470588235294117, 0.1...","(0.6470588235294118, 0.16470588235294117, 0.16...","(0.0, 0.0, 0.0)"
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,"['food', 'fruit', 'baked goods', 'breakfast', ...","[0.9808361530303955, 0.9546540379524231, 0.947...","(0.6588235294117647, 0.4392156862745098, 0.141...",0,"[(0.6470588235294118, 0.16470588235294117, 0.1...","(0.6470588235294118, 0.16470588235294117, 0.16...","(0.6470588235294118, 0.16470588235294117, 0.16..."
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,"['food', 'bread', 'fast food', 'bun', 'america...","[0.9958561658859253, 0.981575608253479, 0.9799...","(0.24313725490196078, 0.12549019607843137, 0.0...",0,"[(0.6470588235294118, 0.16470588235294117, 0.1...","(0.6470588235294118, 0.16470588235294117, 0.16...","(0.6470588235294118, 0.16470588235294117, 0.16..."
4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,"['text', 'font', 'design', 'yellow', 'graphics...","[0.9993002414703369, 0.9640201926231384, 0.951...","(0.792156862745098, 0.7372549019607844, 0.0039...",0,"[(1.0, 1.0, 0.0)]","(1.0, 1.0, 0.0)","(1.0, 1.0, 0.0)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88511,88511,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,"['text', 'food', 'snack']","[0.9999812841415405, 0.9580191373825073, 0.915...","(0.6588235294117647, 0.00784313725490196, 0.71...",0,"[(0.5019607843137255, 0.0, 0.5019607843137255)...","(0.5019607843137255, 0.0, 0.5019607843137255)","(0.5019607843137255, 0.5019607843137255, 0.501..."
88512,88512,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,"['text', 'poster', 'cartoon']","[0.9999863505363464, 0.9184243679046631, 0.910...","(0.01568627450980392, 0.5098039215686274, 0.78...",0,"[(1.0, 1.0, 1.0)]","(1.0, 1.0, 1.0)","(1.0, 1.0, 1.0)"
88513,88513,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,"['text', 'yellow', 'packaging and labeling', '...","[0.9997624158859253, 0.9444339871406555, 0.854...","(0.01568627450980392, 0.2235294117647059, 0.43...",0,"[(1.0, 1.0, 0.0), (0.0, 0.0, 1.0)]","(0.0, 0.0, 1.0)","(1.0, 1.0, 0.0)"
88514,88514,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Crawler error: Redirected to login page,"['sport', 'person', 'barbell', 'physical fitne...","[0.9959811568260193, 0.9808116555213928, 0.971...","(0.6862745098039216, 0.5215686274509804, 0.109...",0,"[(1.0, 1.0, 1.0)]","(1.0, 1.0, 1.0)","(1.0, 1.0, 1.0)"


In [7]:
df.drop(['Unnamed: 0'], axis = 1,inplace=True)

In [8]:
df.drop(df.columns.difference(['tags','confidence_score','accent_color','is_bw','dominant_colors','bg_color','fore_color','likes']), 1, inplace=True)

In [9]:
df

,likes,tags,confidence_score,accent_color,is_bw,dominant_colors,bg_color,fore_color
0,29.0,"['text', 'font', 'graphics', 'screenshot', 'gr...","[0.9980798959732056, 0.9481294751167297, 0.881...","(0.788235294117647, 0.00784313725490196, 0.007...",0,"[(1.0, 0.7529411764705882, 0.796078431372549)]","(1.0, 0.7529411764705882, 0.796078431372549)","(1.0, 0.7529411764705882, 0.796078431372549)"
1,44.0,"['dessert', 'baked goods', 'baking', 'snack', ...","[0.9897554516792297, 0.987897515296936, 0.9828...","(0.6901960784313725, 0.14901960784313725, 0.10...",0,"[(0.6470588235294118, 0.16470588235294117, 0.1...","(0.6470588235294118, 0.16470588235294117, 0.16...","(0.0, 0.0, 0.0)"
2,209.0,"['food', 'fruit', 'baked goods', 'breakfast', ...","[0.9808361530303955, 0.9546540379524231, 0.947...","(0.6588235294117647, 0.4392156862745098, 0.141...",0,"[(0.6470588235294118, 0.16470588235294117, 0.1...","(0.6470588235294118, 0.16470588235294117, 0.16...","(0.6470588235294118, 0.16470588235294117, 0.16..."
3,95.0,"['food', 'bread', 'fast food', 'bun', 'america...","[0.9958561658859253, 0.981575608253479, 0.9799...","(0.24313725490196078, 0.12549019607843137, 0.0...",0,"[(0.6470588235294118, 0.16470588235294117, 0.1...","(0.6470588235294118, 0.16470588235294117, 0.16...","(0.6470588235294118, 0.16470588235294117, 0.16..."
4,101.0,"['text', 'font', 'design', 'yellow', 'graphics...","[0.9993002414703369, 0.9640201926231384, 0.951...","(0.792156862745098, 0.7372549019607844, 0.0039...",0,"[(1.0, 1.0, 0.0)]","(1.0, 1.0, 0.0)","(1.0, 1.0, 0.0)"
...,...,...,...,...,...,...,...,...
88511,8.0,"['text', 'food', 'snack']","[0.9999812841415405, 0.9580191373825073, 0.915...","(0.6588235294117647, 0.00784313725490196, 0.71...",0,"[(0.5019607843137255, 0.0, 0.5019607843137255)...","(0.5019607843137255, 0.0, 0.5019607843137255)","(0.5019607843137255, 0.5019607843137255, 0.501..."
88512,9.0,"['text', 'poster', 'cartoon']","[0.9999863505363464, 0.9184243679046631, 0.910...","(0.01568627450980392, 0.5098039215686274, 0.78...",0,"[(1.0, 1.0, 1.0)]","(1.0, 1.0, 1.0)","(1.0, 1.0, 1.0)"
88513,11.0,"['text', 'yellow', 'packaging and labeling', '...","[0.9997624158859253, 0.9444339871406555, 0.854...","(0.01568627450980392, 0.2235294117647059, 0.43...",0,"[(1.0, 1.0, 0.0), (0.0, 0.0, 1.0)]","(0.0, 0.0, 1.0)","(1.0, 1.0, 0.0)"
88514,NaN,"['sport', 'person', 'barbell', 'physical fitne...","[0.9959811568260193, 0.9808116555213928, 0.971...","(0.6862745098039216, 0.5215686274509804, 0.109...",0,"[(1.0, 1.0, 1.0)]","(1.0, 1.0, 1.0)","(1.0, 1.0, 1.0)"


In [10]:
#calculating missing values in the dataset
missing_values = df.isnull().sum()
missing_per = (missing_values/df.shape[0])*100
missing_table = pd.concat([missing_values,missing_per], axis=1, ignore_index=True) 
missing_table.rename(columns={0:'Total Missing Values',1:'Missing %'}, inplace=True)
missing_table

,Total Missing Values,Missing %
likes,2,0.002259
tags,0,0.000000
confidence_score,0,0.000000
accent_color,0,0.000000
is_bw,0,0.000000
dominant_colors,0,0.000000
bg_color,0,0.000000
fore_color,0,0.000000


In [11]:
#fill the missing values with zero

df['likes'] = df['likes'].fillna(0)

In [13]:
df1=df[:]

In [14]:
REPLACE_BY_SPACE_RE = re.compile('[^a-zA-Z_0-9,.]')

def strip_text(text: str) -> str:
    """
    Removes any left or right spacing (including carriage return) from text.
    Example:
    Input: '  This assignment is cool\n'
    Output: 'This assignment is cool'
    """

    return text.strip()


def replace_br(text: str) -> str:
    """
    Replaces br characters
    """

    return text.replace("'", "")


def replace_special_characters(text: str) -> str:
    """
    Replaces special characters, such as paranthesis,
    with spacing character
    """

    return REPLACE_BY_SPACE_RE.sub(' ', text)


from typing import List, Callable, Dict

PREPROCESSING_PIPELINE = [
                          replace_special_characters,
                          strip_text
                          ]



def text_prepare(text: str, filter_methods: List[Callable[[str], str]] = None) -> str:
    """
    Applies a list of pre-processing functions in sequence (reduce).
    Note that the order is important here!
    """
    filter_methods = filter_methods if filter_methods is not None else PREPROCESSING_PIPELINE
    return reduce(lambda txt, f: f(txt), filter_methods, text)

# Pre-processing

print('Pre-processing text...')
print('[Debug] Before:\n{}'.format(df1.tags[:10]))

L=['tags','confidence_score','accent_color','dominant_colors','bg_color','fore_color']

A=['tags']

for i in L:
  df1[i]=df1[i].apply(lambda txt: text_prepare(str(txt)[1:-1]))



print('[Debug] After:\n{}'.format(df1.tags[:10]))
print()

print("Pre-processing completed!")

Pre-processing text...
[Debug] Before:
0    ['text', 'font', 'graphics', 'screenshot', 'gr...
1    ['dessert', 'baked goods', 'baking', 'snack', ...
2    ['food', 'fruit', 'baked goods', 'breakfast', ...
3    ['food', 'bread', 'fast food', 'bun', 'america...
4    ['text', 'font', 'design', 'yellow', 'graphics...
5    ['snack', 'baked goods', 'baking', 'dessert', ...
6    ['dessert', 'baked goods', 'baking', 'sweetnes...
7    ['text', 'food', 'shelf', 'countertop', 'indoo...
8    ['text', 'table', 'food', 'bakery', 'clothing'...
9    ['baked goods', 'sweetness', 'food', 'baking',...
Name: tags, dtype: object
[Debug] After:
0    text ,  font ,  graphics ,  screenshot ,  grap...
1    dessert ,  baked goods ,  baking ,  snack ,  f...
2    food ,  fruit ,  baked goods ,  breakfast ,  c...
3    food ,  bread ,  fast food ,  bun ,  american ...
4    text ,  font ,  design ,  yellow ,  graphics ,...
5    snack ,  baked goods ,  baking ,  dessert ,  b...
6    dessert ,  baked goods ,  baking , 

In [15]:
df1

,likes,tags,confidence_score,accent_color,is_bw,dominant_colors,bg_color,fore_color
0,29.0,"text , font , graphics , screenshot , grap...","0.9980798959732056, 0.9481294751167297, 0.8818...","0.788235294117647, 0.00784313725490196, 0.0078...",0,"1.0, 0.7529411764705882, 0.796078431372549","1.0, 0.7529411764705882, 0.796078431372549","1.0, 0.7529411764705882, 0.796078431372549"
1,44.0,"dessert , baked goods , baking , snack , f...","0.9897554516792297, 0.987897515296936, 0.98287...","0.6901960784313725, 0.14901960784313725, 0.105...",0,"0.6470588235294118, 0.16470588235294117, 0.164...","0.6470588235294118, 0.16470588235294117, 0.164...","0.0, 0.0, 0.0"
2,209.0,"food , fruit , baked goods , breakfast , c...","0.9808361530303955, 0.9546540379524231, 0.9477...","0.6588235294117647, 0.4392156862745098, 0.1411...",0,"0.6470588235294118, 0.16470588235294117, 0.164...","0.6470588235294118, 0.16470588235294117, 0.164...","0.6470588235294118, 0.16470588235294117, 0.164..."
3,95.0,"food , bread , fast food , bun , american ...","0.9958561658859253, 0.981575608253479, 0.97992...","0.24313725490196078, 0.12549019607843137, 0.07...",0,"0.6470588235294118, 0.16470588235294117, 0.164...","0.6470588235294118, 0.16470588235294117, 0.164...","0.6470588235294118, 0.16470588235294117, 0.164..."
4,101.0,"text , font , design , yellow , graphics ,...","0.9993002414703369, 0.9640201926231384, 0.9513...","0.792156862745098, 0.7372549019607844, 0.00392...",0,"1.0, 1.0, 0.0","1.0, 1.0, 0.0","1.0, 1.0, 0.0"
...,...,...,...,...,...,...,...,...
88511,8.0,"text , food , snack","0.9999812841415405, 0.9580191373825073, 0.9157...","0.6588235294117647, 0.00784313725490196, 0.713...",0,"0.5019607843137255, 0.0, 0.5019607843137255 , ...","0.5019607843137255, 0.0, 0.5019607843137255","0.5019607843137255, 0.5019607843137255, 0.5019..."
88512,9.0,"text , poster , cartoon","0.9999863505363464, 0.9184243679046631, 0.9104...","0.01568627450980392, 0.5098039215686274, 0.780...",0,"1.0, 1.0, 1.0","1.0, 1.0, 1.0","1.0, 1.0, 1.0"
88513,11.0,"text , yellow , packaging and labeling , fo...","0.9997624158859253, 0.9444339871406555, 0.8540...","0.01568627450980392, 0.2235294117647059, 0.439...",0,"1.0, 1.0, 0.0 , 0.0, 0.0, 1.0","0.0, 0.0, 1.0","1.0, 1.0, 0.0"
88514,0.0,"sport , person , barbell , physical fitness...","0.9959811568260193, 0.9808116555213928, 0.9714...","0.6862745098039216, 0.5215686274509804, 0.1098...",0,"1.0, 1.0, 1.0","1.0, 1.0, 1.0","1.0, 1.0, 1.0"


In [16]:
# for i in df1['confidence_score']:

def convert_str2int(i):
  n = []
  y = i.split(', ')
  if len(y[0]) == 0:
    return []
  for j in range(len(y)):
    n.append(float(y[j]))
  return n

df1['confidence_score'] = df1['confidence_score'].apply(convert_str2int)
df1['accent_color'] = df1['accent_color'].apply(convert_str2int)
df1['dominant_colors'] = df1['dominant_colors'].apply(convert_str2int)
df1['bg_color'] = df1['bg_color'].apply(convert_str2int)
df1['fore_color'] = df1['fore_color'].apply(convert_str2int)

In [17]:
df1

,likes,tags,confidence_score,accent_color,is_bw,dominant_colors,bg_color,fore_color
0,29.0,"text , font , graphics , screenshot , grap...","[0.9980798959732056, 0.9481294751167297, 0.881...","[0.788235294117647, 0.00784313725490196, 0.007...",0,"[1.0, 0.7529411764705882, 0.796078431372549]","[1.0, 0.7529411764705882, 0.796078431372549]","[1.0, 0.7529411764705882, 0.796078431372549]"
1,44.0,"dessert , baked goods , baking , snack , f...","[0.9897554516792297, 0.987897515296936, 0.9828...","[0.6901960784313725, 0.14901960784313725, 0.10...",0,"[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...","[0.0, 0.0, 0.0]"
2,209.0,"food , fruit , baked goods , breakfast , c...","[0.9808361530303955, 0.9546540379524231, 0.947...","[0.6588235294117647, 0.4392156862745098, 0.141...",0,"[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16..."
3,95.0,"food , bread , fast food , bun , american ...","[0.9958561658859253, 0.981575608253479, 0.9799...","[0.24313725490196078, 0.12549019607843137, 0.0...",0,"[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16..."
4,101.0,"text , font , design , yellow , graphics ,...","[0.9993002414703369, 0.9640201926231384, 0.951...","[0.792156862745098, 0.7372549019607844, 0.0039...",0,"[1.0, 1.0, 0.0]","[1.0, 1.0, 0.0]","[1.0, 1.0, 0.0]"
...,...,...,...,...,...,...,...,...
88511,8.0,"text , food , snack","[0.9999812841415405, 0.9580191373825073, 0.915...","[0.6588235294117647, 0.00784313725490196, 0.71...",0,"[0.5019607843137255, 0.0, 0.5019607843137255, ...","[0.5019607843137255, 0.0, 0.5019607843137255]","[0.5019607843137255, 0.5019607843137255, 0.501..."
88512,9.0,"text , poster , cartoon","[0.9999863505363464, 0.9184243679046631, 0.910...","[0.01568627450980392, 0.5098039215686274, 0.78...",0,"[1.0, 1.0, 1.0]","[1.0, 1.0, 1.0]","[1.0, 1.0, 1.0]"
88513,11.0,"text , yellow , packaging and labeling , fo...","[0.9997624158859253, 0.9444339871406555, 0.854...","[0.01568627450980392, 0.2235294117647059, 0.43...",0,"[1.0, 1.0, 0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 1.0]","[1.0, 1.0, 0.0]"
88514,0.0,"sport , person , barbell , physical fitness...","[0.9959811568260193, 0.9808116555213928, 0.971...","[0.6862745098039216, 0.5215686274509804, 0.109...",0,"[1.0, 1.0, 1.0]","[1.0, 1.0, 1.0]","[1.0, 1.0, 1.0]"


In [18]:
def convert_str2list(i):
  n = []
  y = i.split(', ')
  for j in range(len(y)):
    n.append(str(y[j]))
  return n
df1['tags'] = df['tags'].apply(convert_str2list)
df1['tags'][0]

['text ',
 ' font ',
 ' graphics ',
 ' screenshot ',
 ' graphic design ',
 ' design ',
 ' typography']

In [19]:
df1['confidence_score'][6]

[0.9938297271728516,
 0.9784528613090515,
 0.977806806564331,
 0.966992974281311,
 0.9616279602050781,
 0.9515609741210938,
 0.9280351400375366,
 0.912075936794281,
 0.9083282947540283,
 0.9079588651657104,
 0.8951475620269775,
 0.8874362707138062,
 0.8836393356323242,
 0.8746436834335327,
 0.8713182210922241,
 0.8668385744094849,
 0.8426526784896851,
 0.786042332649231,
 0.5681669116020203]

In [20]:
def trim_lowConfidence_tags(tag_column):
  for scoreList in df1['confidence_score']:
    for score_index in range(len(scoreList)):
      if scoreList[score_index]<0.5:
        return tag_column[:score_index]

df1['tags'] = df1['tags'].apply(trim_lowConfidence_tags)  

In [21]:
df1.drop('confidence_score',1,inplace=True)

In [22]:
df1

,likes,tags,accent_color,is_bw,dominant_colors,bg_color,fore_color
0,29.0,"[text , font , graphics , screenshot , gra...","[0.788235294117647, 0.00784313725490196, 0.007...",0,"[1.0, 0.7529411764705882, 0.796078431372549]","[1.0, 0.7529411764705882, 0.796078431372549]","[1.0, 0.7529411764705882, 0.796078431372549]"
1,44.0,"[dessert , baked goods , baking , snack , ...","[0.6901960784313725, 0.14901960784313725, 0.10...",0,"[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...","[0.0, 0.0, 0.0]"
2,209.0,"[food , fruit , baked goods , breakfast , ...","[0.6588235294117647, 0.4392156862745098, 0.141...",0,"[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16..."
3,95.0,"[food , bread , fast food , bun , american...","[0.24313725490196078, 0.12549019607843137, 0.0...",0,"[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16..."
4,101.0,"[text , font , design , yellow , graphics ...","[0.792156862745098, 0.7372549019607844, 0.0039...",0,"[1.0, 1.0, 0.0]","[1.0, 1.0, 0.0]","[1.0, 1.0, 0.0]"
...,...,...,...,...,...,...,...
88511,8.0,"[text , food , snack]","[0.6588235294117647, 0.00784313725490196, 0.71...",0,"[0.5019607843137255, 0.0, 0.5019607843137255, ...","[0.5019607843137255, 0.0, 0.5019607843137255]","[0.5019607843137255, 0.5019607843137255, 0.501..."
88512,9.0,"[text , poster , cartoon]","[0.01568627450980392, 0.5098039215686274, 0.78...",0,"[1.0, 1.0, 1.0]","[1.0, 1.0, 1.0]","[1.0, 1.0, 1.0]"
88513,11.0,"[text , yellow , packaging and labeling , f...","[0.01568627450980392, 0.2235294117647059, 0.43...",0,"[1.0, 1.0, 0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 1.0]","[1.0, 1.0, 0.0]"
88514,0.0,"[sport , person , barbell , physical fitnes...","[0.6862745098039216, 0.5215686274509804, 0.109...",0,"[1.0, 1.0, 1.0]","[1.0, 1.0, 1.0]","[1.0, 1.0, 1.0]"


In [23]:
df1['dominant_colors'][:3]

0         [1.0, 0.7529411764705882, 0.796078431372549]
1    [0.6470588235294118, 0.16470588235294117, 0.16...
2    [0.6470588235294118, 0.16470588235294117, 0.16...
Name: dominant_colors, dtype: object

In [24]:
def separate_column(column):
  if len(column)>0:
    return column[0]
  else:
    return []

df1['dominant_colors_R'] = df1['dominant_colors'].apply(separate_column)  
df1['accent_color_R'] = df1['accent_color'].apply(separate_column)  
df1['bg_color_R'] = df1['bg_color'].apply(separate_column)  
df1['fore_color_R'] = df1['fore_color'].apply(separate_column)  

In [25]:
def separate_column(column):
  if len(column)>0:
    return column[1]
  else:
    return []

df1['dominant_colors_G'] = df1['dominant_colors'].apply(separate_column)  
df1['accent_color_G'] = df1['accent_color'].apply(separate_column)  
df1['bg_color_G'] = df1['bg_color'].apply(separate_column)  
df1['fore_color_G'] = df1['fore_color'].apply(separate_column)  

In [26]:
def separate_column(column):
  if len(column)>0:
    return column[2]
  else:
    return []

df1['dominant_colors_B'] = df1['dominant_colors'].apply(separate_column)  
df1['accent_color_B'] = df1['accent_color'].apply(separate_column)  
df1['bg_color_B'] = df1['bg_color'].apply(separate_column)  
df1['fore_color_B'] = df1['fore_color'].apply(separate_column)  

In [27]:
df1

,likes,tags,accent_color,is_bw,dominant_colors,bg_color,fore_color,dominant_colors_R,accent_color_R,bg_color_R,fore_color_R,dominant_colors_G,accent_color_G,bg_color_G,fore_color_G,dominant_colors_B,accent_color_B,bg_color_B,fore_color_B
0,29.0,"[text , font , graphics , screenshot , gra...","[0.788235294117647, 0.00784313725490196, 0.007...",0,"[1.0, 0.7529411764705882, 0.796078431372549]","[1.0, 0.7529411764705882, 0.796078431372549]","[1.0, 0.7529411764705882, 0.796078431372549]",1,0.788235,1.000000,1.000000,0.752941,0.007843,0.752941,0.752941,0.796078,0.007843,0.796078,0.796078
1,44.0,"[dessert , baked goods , baking , snack , ...","[0.6901960784313725, 0.14901960784313725, 0.10...",0,"[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...","[0.0, 0.0, 0.0]",0.647059,0.690196,0.647059,0.000000,0.164706,0.149020,0.164706,0.000000,0.164706,0.105882,0.164706,0.000000
2,209.0,"[food , fruit , baked goods , breakfast , ...","[0.6588235294117647, 0.4392156862745098, 0.141...",0,"[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...",0.647059,0.658824,0.647059,0.647059,0.164706,0.439216,0.164706,0.164706,0.164706,0.141176,0.164706,0.164706
3,95.0,"[food , bread , fast food , bun , american...","[0.24313725490196078, 0.12549019607843137, 0.0...",0,"[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...",0.647059,0.243137,0.647059,0.647059,0.164706,0.125490,0.164706,0.164706,0.164706,0.070588,0.164706,0.164706
4,101.0,"[text , font , design , yellow , graphics ...","[0.792156862745098, 0.7372549019607844, 0.0039...",0,"[1.0, 1.0, 0.0]","[1.0, 1.0, 0.0]","[1.0, 1.0, 0.0]",1,0.792157,1.000000,1.000000,1,0.737255,1.000000,1.000000,0,0.003922,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88511,8.0,"[text , food , snack]","[0.6588235294117647, 0.00784313725490196, 0.71...",0,"[0.5019607843137255, 0.0, 0.5019607843137255, ...","[0.5019607843137255, 0.0, 0.5019607843137255]","[0.5019607843137255, 0.5019607843137255, 0.501...",0.501961,0.658824,0.501961,0.501961,0,0.007843,0.000000,0.501961,0.501961,0.713725,0.501961,0.501961
88512,9.0,"[text , poster , cartoon]","[0.01568627450980392, 0.5098039215686274, 0.78...",0,"[1.0, 1.0, 1.0]","[1.0, 1.0, 1.0]","[1.0, 1.0, 1.0]",1,0.015686,1.000000,1.000000,1,0.509804,1.000000,1.000000,1,0.780392,1.000000,1.000000
88513,11.0,"[text , yellow , packaging and labeling , f...","[0.01568627450980392, 0.2235294117647059, 0.43...",0,"[1.0, 1.0, 0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 1.0]","[1.0, 1.0, 0.0]",1,0.015686,0.000000,1.000000,1,0.223529,0.000000,1.000000,0,0.439216,1.000000,0.000000
88514,0.0,"[sport , person , barbell , physical fitnes...","[0.6862745098039216, 0.5215686274509804, 0.109...",0,"[1.0, 1.0, 1.0]","[1.0, 1.0, 1.0]","[1.0, 1.0, 1.0]",1,0.686275,1.000000,1.000000,1,0.521569,1.000000,1.000000,1,0.109804,1.000000,1.000000


In [28]:
dropped = ['accent_color', 'dominant_colors', 'bg_color','fore_color']
data = df1.drop(dropped,axis=1)

In [29]:
data

,likes,tags,is_bw,dominant_colors_R,accent_color_R,bg_color_R,fore_color_R,dominant_colors_G,accent_color_G,bg_color_G,fore_color_G,dominant_colors_B,accent_color_B,bg_color_B,fore_color_B
0,29.0,"[text , font , graphics , screenshot , gra...",0,1,0.788235,1.000000,1.000000,0.752941,0.007843,0.752941,0.752941,0.796078,0.007843,0.796078,0.796078
1,44.0,"[dessert , baked goods , baking , snack , ...",0,0.647059,0.690196,0.647059,0.000000,0.164706,0.149020,0.164706,0.000000,0.164706,0.105882,0.164706,0.000000
2,209.0,"[food , fruit , baked goods , breakfast , ...",0,0.647059,0.658824,0.647059,0.647059,0.164706,0.439216,0.164706,0.164706,0.164706,0.141176,0.164706,0.164706
3,95.0,"[food , bread , fast food , bun , american...",0,0.647059,0.243137,0.647059,0.647059,0.164706,0.125490,0.164706,0.164706,0.164706,0.070588,0.164706,0.164706
4,101.0,"[text , font , design , yellow , graphics ...",0,1,0.792157,1.000000,1.000000,1,0.737255,1.000000,1.000000,0,0.003922,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88511,8.0,"[text , food , snack]",0,0.501961,0.658824,0.501961,0.501961,0,0.007843,0.000000,0.501961,0.501961,0.713725,0.501961,0.501961
88512,9.0,"[text , poster , cartoon]",0,1,0.015686,1.000000,1.000000,1,0.509804,1.000000,1.000000,1,0.780392,1.000000,1.000000
88513,11.0,"[text , yellow , packaging and labeling , f...",0,1,0.015686,0.000000,1.000000,1,0.223529,0.000000,1.000000,0,0.439216,1.000000,0.000000
88514,0.0,"[sport , person , barbell , physical fitnes...",0,1,0.686275,1.000000,1.000000,1,0.521569,1.000000,1.000000,1,0.109804,1.000000,1.000000


In [30]:
data2=data.drop('tags',1)

In [31]:
data2

,likes,is_bw,dominant_colors_R,accent_color_R,bg_color_R,fore_color_R,dominant_colors_G,accent_color_G,bg_color_G,fore_color_G,dominant_colors_B,accent_color_B,bg_color_B,fore_color_B
0,29.0,0,1,0.788235,1.000000,1.000000,0.752941,0.007843,0.752941,0.752941,0.796078,0.007843,0.796078,0.796078
1,44.0,0,0.647059,0.690196,0.647059,0.000000,0.164706,0.149020,0.164706,0.000000,0.164706,0.105882,0.164706,0.000000
2,209.0,0,0.647059,0.658824,0.647059,0.647059,0.164706,0.439216,0.164706,0.164706,0.164706,0.141176,0.164706,0.164706
3,95.0,0,0.647059,0.243137,0.647059,0.647059,0.164706,0.125490,0.164706,0.164706,0.164706,0.070588,0.164706,0.164706
4,101.0,0,1,0.792157,1.000000,1.000000,1,0.737255,1.000000,1.000000,0,0.003922,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88511,8.0,0,0.501961,0.658824,0.501961,0.501961,0,0.007843,0.000000,0.501961,0.501961,0.713725,0.501961,0.501961
88512,9.0,0,1,0.015686,1.000000,1.000000,1,0.509804,1.000000,1.000000,1,0.780392,1.000000,1.000000
88513,11.0,0,1,0.015686,0.000000,1.000000,1,0.223529,0.000000,1.000000,0,0.439216,1.000000,0.000000
88514,0.0,0,1,0.686275,1.000000,1.000000,1,0.521569,1.000000,1.000000,1,0.109804,1.000000,1.000000


In [32]:
data['tags']=data['tags'].apply(lambda txt: text_prepare(str(txt)))

In [33]:
tokenized_tag = data['tags'].apply(lambda x: x.split())
tokenized_tag[0]

['text',
 ',',
 'font',
 ',',
 'graphics',
 ',',
 'screenshot',
 ',',
 'graphic',
 'design',
 ',',
 'design',
 ',',
 'typography']

##Word2vec

### Training for skip gram

In [35]:
model_w2v_skipgram = gensim.models.Word2Vec(tokenized_tag,
            size=200, # desired no. of features/independent variables
            window=5, # context window size
            min_count=2, # Ignores all words with total frequency lower than 2.                                  
            sg = 1, # 1 for skip-gram model
            hs = 0,
            negative = 10, # for negative sampling
            workers= 32, # no.of cores 
            )
model_w2v_skipgram.train(tokenized_tag, total_examples= len(tokenized_tag), epochs=20)

INFO - 19:40:54: collecting all words and their counts
INFO - 19:40:54: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 19:40:54: PROGRESS: at sentence #10000, processed 204860 words, keeping 2339 word types
INFO - 19:40:54: PROGRESS: at sentence #20000, processed 415054 words, keeping 2816 word types
INFO - 19:40:54: PROGRESS: at sentence #30000, processed 622777 words, keeping 3089 word types
INFO - 19:40:54: PROGRESS: at sentence #40000, processed 834536 words, keeping 3287 word types
INFO - 19:40:55: PROGRESS: at sentence #50000, processed 1041326 words, keeping 3425 word types
INFO - 19:40:55: PROGRESS: at sentence #60000, processed 1253866 words, keeping 3563 word types
INFO - 19:40:55: PROGRESS: at sentence #70000, processed 1462839 words, keeping 3664 word types
INFO - 19:40:55: PROGRESS: at sentence #80000, processed 1672957 words, keeping 3777 word types
INFO - 19:40:55: collected 3831 word types from a corpus of 1850422 raw words and 88516 sentences


(16940754, 37008440)

In [36]:
model_w2v_skipgram.save(path + "Save Models\\200_word2vec_skipgram_.model")

INFO - 19:42:27: saving Word2Vec object under C:\Users\jyoti\Desktop\NLP\NLP-project-\Save Models\200_word2vec_skipgram_.model, separately None
INFO - 19:42:27: not storing attribute vectors_norm
INFO - 19:42:27: not storing attribute cum_table
INFO - 19:42:28: saved C:\Users\jyoti\Desktop\NLP\NLP-project-\Save Models\200_word2vec_skipgram_.model


In [37]:
model_w2v_skipgram.wv.most_similar("Gastroenteritis")

INFO - 19:42:34: precomputing L2-norms of word weight vectors


KeyError: "word 'Gastroenteritis' not in vocabulary"

### Training for CBOW 

In [41]:
model_w2v_CBOW = gensim.models.Word2Vec(tokenized_tag,
            size=200, # desired no. of features/independent variables
            window=5, # context window size
            min_count=2, # Ignores all words with total frequency lower than 2.                                  
            sg = 0, # 0 for CBOW model
            hs = 0,
            negative = 10, # for negative sampling
            workers= 32, # no.of cores
            seed = 34
)
model_w2v_CBOW.train(tokenized_tag, total_examples= len(tokenized_tag), epochs=20)

INFO - 19:43:56: collecting all words and their counts
INFO - 19:43:56: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 19:43:57: PROGRESS: at sentence #10000, processed 204860 words, keeping 2339 word types
INFO - 19:43:57: PROGRESS: at sentence #20000, processed 415054 words, keeping 2816 word types
INFO - 19:43:57: PROGRESS: at sentence #30000, processed 622777 words, keeping 3089 word types
INFO - 19:43:57: PROGRESS: at sentence #40000, processed 834536 words, keeping 3287 word types
INFO - 19:43:57: PROGRESS: at sentence #50000, processed 1041326 words, keeping 3425 word types
INFO - 19:43:57: PROGRESS: at sentence #60000, processed 1253866 words, keeping 3563 word types
INFO - 19:43:57: PROGRESS: at sentence #70000, processed 1462839 words, keeping 3664 word types
INFO - 19:43:58: PROGRESS: at sentence #80000, processed 1672957 words, keeping 3777 word types
INFO - 19:43:58: collected 3831 word types from a corpus of 1850422 raw words and 88516 sentences


(16942086, 37008440)

In [44]:
model_w2v_CBOW.save(path + "Save Models//200_word2vec_CBOW_.model")

INFO - 19:44:49: saving Word2Vec object under C:\Users\jyoti\Desktop\NLP\NLP-project-\Save Models//200_word2vec_CBOW_.model, separately None
INFO - 19:44:49: not storing attribute vectors_norm
INFO - 19:44:49: not storing attribute cum_table
INFO - 19:44:49: saved C:\Users\jyoti\Desktop\NLP\NLP-project-\Save Models//200_word2vec_CBOW_.model


In [45]:
model_w2v_CBOW.wv.most_similar("Gastroenteritis")

KeyError: "word 'Gastroenteritis' not in vocabulary"

In [46]:
model_w2v_CBOW.wv.most_similar("bread")

[('runza', 0.6190930604934692),
 ('cinnamon', 0.5607609748840332),
 ('loaf', 0.49205243587493896),
 ('grain', 0.47905433177948),
 ('wheat', 0.4658154249191284),
 ('staple', 0.46487993001937866),
 ('sourdough', 0.4505365788936615),
 ('banh', 0.4489794373512268),
 ('food', 0.43922916054725647),
 ('breakfast', 0.43278443813323975)]

##FastText

In [48]:
modelFastTextSkipGram = FastText(tokenized_tag, 
                     size=200, # desired no. of features/independent variables
                     window=2,  # context window size
                     min_count=2, # Ignores all words with total frequency lower than 2.  
                     workers=32, # no.of cores
                     hs = 0,
                     negative = 10, # for negative sampling
                     sg=1  # 1 for Skipgram model
                     )

modelFastTextSkipGram.build_vocab(tokenized_tag, update=True)
modelFastTextSkipGram.train(tokenized_tag, total_examples= len(tokenized_tag), epochs=20)

INFO - 19:45:03: collecting all words and their counts
INFO - 19:45:03: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 19:45:03: PROGRESS: at sentence #10000, processed 204860 words, keeping 2339 word types
INFO - 19:45:03: PROGRESS: at sentence #20000, processed 415054 words, keeping 2816 word types
INFO - 19:45:04: PROGRESS: at sentence #30000, processed 622777 words, keeping 3089 word types
INFO - 19:45:04: PROGRESS: at sentence #40000, processed 834536 words, keeping 3287 word types
INFO - 19:45:04: PROGRESS: at sentence #50000, processed 1041326 words, keeping 3425 word types
INFO - 19:45:04: PROGRESS: at sentence #60000, processed 1253866 words, keeping 3563 word types
INFO - 19:45:04: PROGRESS: at sentence #70000, processed 1462839 words, keeping 3664 word types
INFO - 19:45:05: PROGRESS: at sentence #80000, processed 1672957 words, keeping 3777 word types
INFO - 19:45:05: collected 3831 word types from a corpus of 1850422 raw words and 88516 sentences


In [49]:
modelFastTextSkipGram.save(path+ "Save Models\\200_fastText_Skipgram_.model")

INFO - 19:50:21: saving FastText object under C:\Users\jyoti\Desktop\NLP\NLP-project-\Save Models\200_fastText_Skipgram_.model, separately None
INFO - 19:50:21: not storing attribute vectors_norm
INFO - 19:50:21: not storing attribute vectors_vocab_norm
INFO - 19:50:21: not storing attribute vectors_ngrams_norm
INFO - 19:50:21: not storing attribute buckets_word
INFO - 19:50:22: saved C:\Users\jyoti\Desktop\NLP\NLP-project-\Save Models\200_fastText_Skipgram_.model


In [50]:
modelFastTextSkipGram.wv.most_similar("Gastroenteritis")

INFO - 19:50:23: precomputing L2-norms of word weight vectors
INFO - 19:50:24: precomputing L2-norms of ngram weight vectors


[('jitsu', 0.7175878286361694),
 ('3d', 0.6969789266586304),
 ('x', 0.684368908405304),
 ('celup', 0.6838890910148621),
 ('jiu', 0.6789247989654541),
 ('vinyl', 0.6783366203308105),
 ('analgesic', 0.6767390370368958),
 ('tagliatelle', 0.6721645593643188),
 ('chalupa', 0.6689993143081665),
 ('astronomy', 0.664101779460907)]

In [51]:
modelFastTextSkipGram.wv.most_similar("pizza")

[('flatbread', 0.7170838713645935),
 ('pepperoni', 0.6624626517295837),
 ('manakish', 0.6588062047958374),
 ('italian', 0.6555225849151611),
 ('casserole', 0.6527655124664307),
 ('california', 0.635886549949646),
 ('sicilian', 0.6321481466293335),
 ('quiche', 0.6232922673225403),
 ('cassolette', 0.5883208513259888),
 ('macaroni', 0.5702067613601685)]

In [54]:
modelFastTextCBOW = FastText(tokenized_tag, 
                     size=200, # desired no. of features/independent variables
                     window=2,  # context window size
                     min_count=2, # Ignores all words with total frequency lower than 2  
                     workers=32, # no.of cores
                     hs = 0,
                     negative = 10, # for negative sampling
                     sg=0  # 0 for CBOW model
                     )
modelFastTextCBOW.build_vocab(tokenized_tag, update=True)
modelFastTextCBOW.train(tokenized_tag, total_examples= len(tokenized_tag), epochs=20)

INFO - 19:52:00: collecting all words and their counts
INFO - 19:52:00: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 19:52:00: PROGRESS: at sentence #10000, processed 204860 words, keeping 2339 word types
INFO - 19:52:00: PROGRESS: at sentence #20000, processed 415054 words, keeping 2816 word types
INFO - 19:52:00: PROGRESS: at sentence #30000, processed 622777 words, keeping 3089 word types
INFO - 19:52:00: PROGRESS: at sentence #40000, processed 834536 words, keeping 3287 word types
INFO - 19:52:00: PROGRESS: at sentence #50000, processed 1041326 words, keeping 3425 word types
INFO - 19:52:00: PROGRESS: at sentence #60000, processed 1253866 words, keeping 3563 word types
INFO - 19:52:01: PROGRESS: at sentence #70000, processed 1462839 words, keeping 3664 word types
INFO - 19:52:01: PROGRESS: at sentence #80000, processed 1672957 words, keeping 3777 word types
INFO - 19:52:01: collected 3831 word types from a corpus of 1850422 raw words and 88516 sentences


In [55]:
modelFastTextCBOW.save(path+ "Save Models//200_fastText_CBOW_.model")

INFO - 19:59:15: saving FastText object under C:\Users\jyoti\Desktop\NLP\NLP-project-\Save Models//200_fastText_CBOW_.model, separately None
INFO - 19:59:15: not storing attribute vectors_norm
INFO - 19:59:15: not storing attribute vectors_vocab_norm
INFO - 19:59:15: not storing attribute vectors_ngrams_norm
INFO - 19:59:15: not storing attribute buckets_word
INFO - 19:59:16: saved C:\Users\jyoti\Desktop\NLP\NLP-project-\Save Models//200_fastText_CBOW_.model


In [56]:
modelFastTextSkipGram.wv.most_similar("Gastroenteritis")

[('jitsu', 0.7175878286361694),
 ('3d', 0.6969789266586304),
 ('x', 0.684368908405304),
 ('celup', 0.6838890910148621),
 ('jiu', 0.6789247989654541),
 ('vinyl', 0.6783366203308105),
 ('analgesic', 0.6767390370368958),
 ('tagliatelle', 0.6721645593643188),
 ('chalupa', 0.6689993143081665),
 ('astronomy', 0.664101779460907)]

In [57]:
def word_vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0
    for word in tokens:
        try:
            vec += model_w2v_skipgram[word].reshape((1, size))
            count += 1.
        except KeyError:
            continue
    if count != 0:
        vec /= count
    return vec

In [58]:
wordvec_arrays = np.zeros((len(tokenized_tag), 200)) 
for i in range(len(tokenized_tag)):
    wordvec_arrays[i,:] = word_vector(tokenized_tag[i], 200)
wordvec_df = pd.DataFrame(wordvec_arrays)
wordvec_df.shape

C:\Users\jyoti\anaconda3\envs\nlp\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


(88516, 200)

In [59]:
dataset_word2vec = pd.DataFrame(np.hstack((wordvec_df,data2)))

In [60]:
dataset_word2vec[200]

0         29
1         44
2        209
3         95
4        101
        ... 
88511      8
88512      9
88513     11
88514      0
88515      0
Name: 200, Length: 88516, dtype: object

##doc2vec

In [61]:
from gensim.models.doc2vec import LabeledSentence

def add_label(twt):
    output = []
    for i, s in zip(twt.index, twt):
        output.append(LabeledSentence(s, ["tags_" + str(i)]))
    return output

labeled_tags = add_label(tokenized_tag) # label all the tweets

C:\Users\jyoti\anaconda3\envs\nlp\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  


### doc2vec

In [62]:
model_d2vec = gensim.models.Doc2Vec(dm=1, # dm = 1 for ‘distributed memory’ model
                                  dm_mean=1, # dm_mean = 1 for using mean of the context word vectors
                                  vector_size=200, # no. of desired features
                                  window=5, # width of the context window                                  
                                  negative=7, # if > 0 then negative sampling will be used
                                  min_count=5, # Ignores all words with total frequency lower than 5.                                  
                                  workers=32, # no. of cores                                  
                                  alpha=0.1, # learning rate                                  

                                 ) 
model_d2vec.build_vocab([i for i in tqdm(labeled_tags)])
model_d2vec.train(labeled_tags, total_examples= len(tokenized_tag), epochs=15)

100%|██████████| 88516/88516 [00:00<00:00, 842927.07it/s]
INFO - 20:01:22: collecting all words and their counts
INFO - 20:01:22: PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
INFO - 20:01:22: PROGRESS: at example #10000, processed 204860 words (1052607/s), 2339 word types, 10000 tags
INFO - 20:01:22: PROGRESS: at example #20000, processed 415054 words (359907/s), 2816 word types, 20000 tags
INFO - 20:01:23: PROGRESS: at example #30000, processed 622777 words (1082226/s), 3089 word types, 30000 tags
INFO - 20:01:23: PROGRESS: at example #40000, processed 834536 words (1060523/s), 3287 word types, 40000 tags
INFO - 20:01:23: PROGRESS: at example #50000, processed 1041326 words (685160/s), 3425 word types, 50000 tags
INFO - 20:01:23: PROGRESS: at example #60000, processed 1253866 words (613717/s), 3563 word types, 60000 tags
INFO - 20:01:24: PROGRESS: at example #70000, processed 1462839 words (807346/s), 3664 word types, 70000 tags
INFO - 20:01:24: PROGRESS: at 

In [63]:
model_d2vec.save(path +"Save Models//200_doc2vec_.model")

INFO - 20:06:47: saving Doc2Vec object under C:\Users\jyoti\Desktop\NLP\NLP-project-\Save Models//200_doc2vec_.model, separately None
INFO - 20:06:47: storing np array 'vectors_docs' to C:\Users\jyoti\Desktop\NLP\NLP-project-\Save Models//200_doc2vec_.model.docvecs.vectors_docs.npy
INFO - 20:06:47: saved C:\Users\jyoti\Desktop\NLP\NLP-project-\Save Models//200_doc2vec_.model


###doc2vec CBOW

In [64]:
docvec_arrays = np.zeros((len(tokenized_tag), 200)) 
for i in range(len(df1)):
    docvec_arrays[i,:] = model_d2vec.docvecs[i].reshape((1,200))    
docvec_df = pd.DataFrame(docvec_arrays) 
docvec_df.shape

(88516, 200)

In [65]:
dataset_doc2vec = pd.DataFrame(np.hstack((docvec_df,data2)))
dataset_doc2vec

,0,1,2,3,4,5,6,7,8,9,...,204,205,206,207,208,209,210,211,212,213
0,0.00799415,-0.107767,0.131595,-0.115951,-0.0641469,-0.261071,0.0920882,0.038032,0.0272573,0.041599,...,1,1,0.752941,0.00784314,0.752941,0.752941,0.796078,0.00784314,0.796078,0.796078
1,-0.00414995,-0.394838,0.303378,-0.25046,-0.177917,-0.0210222,0.371384,-0.0273701,0.132664,-0.00688097,...,0.647059,0,0.164706,0.14902,0.164706,0,0.164706,0.105882,0.164706,0
2,0.503836,0.334134,-0.431602,0.490139,-0.156971,-0.458985,-0.107672,-0.320003,0.0652374,-0.138921,...,0.647059,0.647059,0.164706,0.439216,0.164706,0.164706,0.164706,0.141176,0.164706,0.164706
3,0.238243,-0.246419,0.309907,-0.200666,0.0289534,-0.106091,0.201354,-0.0890602,-0.143608,0.56232,...,0.647059,0.647059,0.164706,0.12549,0.164706,0.164706,0.164706,0.0705882,0.164706,0.164706
4,-0.155593,-0.0741085,0.368026,0.253695,0.0672224,-0.0958057,-0.161886,-0.0759832,-0.117824,-0.152183,...,1,1,1,0.737255,1,1,0,0.00392157,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88511,0.0426955,-0.166736,0.0721459,-0.164703,-0.0413829,-0.0506933,0.221393,-0.0432113,0.161843,-0.0269484,...,0.501961,0.501961,0,0.00784314,0,0.501961,0.501961,0.713725,0.501961,0.501961
88512,-0.23814,-0.169677,0.288291,-0.240618,-0.0380192,-0.215806,-0.0044513,-0.0407771,-0.0664522,-0.0913228,...,1,1,1,0.509804,1,1,1,0.780392,1,1
88513,0.289867,-0.00213439,-0.227961,-0.064285,-0.0110838,-0.296191,-0.259062,-0.209015,0.154042,-0.119695,...,0,1,1,0.223529,0,1,0,0.439216,1,0
88514,0.13463,-0.0485432,0.0280303,0.0332373,0.0447504,-0.135705,0.218476,0.0673653,-0.00943613,-0.179415,...,1,1,1,0.521569,1,1,1,0.109804,1,1


## Train test split

In [66]:
y=dataset_word2vec[200]
X1=dataset_word2vec.drop(200,1)

In [67]:
x_train, x_test, y_train, y_test = train_test_split(X1, y, test_size=0.2, random_state=2022)

###XGBoost model

In [68]:
#XGBoost hyper-parameter tuning
def hyperParameterTuning(X_train, y_train):
    param_tuning = {
        'learning_rate': [0.01, 0.1],
        'max_depth': [3, 5, 7, 10],
        'min_child_weight': [1, 3, 5],
        'subsample': [0.5, 0.7],
        'colsample_bytree': [0.5, 0.7],
        'n_estimators' : [100, 200, 500],
        'objective': ['reg:squarederror']
    }

    xgb_model = XGBRegressor()

    gsearch = GridSearchCV(estimator = xgb_model,
                           param_grid = param_tuning,                        
                           #scoring = 'neg_mean_absolute_error', #MAE
                           #scoring = 'neg_mean_squared_error',  #MSE
                           cv = 5,
                           n_jobs = -1,
                           verbose = 1)

    gsearch.fit(X_train,y_train)

    return gsearch.best_params_

In [70]:
from xgboost import XGBRegressor

xgb_model = XGBRegressor(
        objective = 'reg:squarederror',
        colsample_bytree = 0.5,
        learning_rate = 0.05,
        max_depth = 6,
        min_child_weight = 1,
        n_estimators = 1000,
        subsample = 0.7)

%time xgb_model.fit(list(x_train), y_train, early_stopping_rounds=5, verbose=False).ravel()

y_pred_xgb = xgb_model.predict(x_test)

ValueError: ('Expecting 2 dimensional numpy.ndarray, got: ', (213,))

NotFittedError: need to call fit or load_model beforehand

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred_xgb))
print("RMSE: %f" % (rmse))
R2=r2_score(y_test,y_pred_xgb)
print("R2 Score: %f" % (R2))
print()

In [ ]:
def plot_feature_importance(importance,names,model_type):

#Create arrays from feature importance and feature names
  feature_importance = np.array(importance)
  feature_names = np.array(names)

  #Create a DataFrame using a Dictionary
  data={'feature_names':feature_names,'feature_importance':feature_importance}
  fi_df = pd.DataFrame(data)

  #Sort the DataFrame in order decreasing feature importance
  fi_df.sort_values(by=['feature_importance'], ascending=False,inplace=True)

  #Define size of bar plot
  plt.figure(figsize=(20,20))
  #Plot Searborn bar chart
  sns.barplot(x=fi_df['feature_importance'], y=fi_df['feature_names'])
  #Add chart labels
  plt.title(model_type + 'FEATURE IMPORTANCE')
  plt.xlabel('FEATURE IMPORTANCE')
  plt.ylabel('FEATURE NAMES')

plot_feature_importance(xg_reg.feature_importances_,x_train.columns,'XG BOOST')